In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import json
import cv2
from matplotlib import pyplot as plt

In [3]:
def return_coordinates(json_obj):
    obj=json_obj['objects']
    coordinate_list=[]
    for bbx in obj:
        coordinates=bbx['relative_coordinates']
        center_x,center_y,width,height=coordinates['center_x'],coordinates['center_y'],coordinates['width'],coordinates['height']
        confidence=bbx['confidence']
        coordinate_list.append((center_x,center_y,width,height,confidence))
    return coordinate_list

In [4]:
def return_image_path(json_obj):
    path=json_obj['filename']
    return path

In [5]:
def return_image_size(img):
  return img.shape[1],img.shape[0]

In [6]:
def return_image_name(path):
  image_name=path[40:]
  return image_name

In [7]:
def return_json_name(img_name):
  JSON_DIR="/content/drive/MyDrive/model/ann/"
  return JSON_DIR+img_name[:-3]+"png"+".json"

In [8]:
def prepare_cropped_image(image):
      image = Image.fromarray(image, 'RGB')
      image = image.resize((32, 32))
      np_image=np.array(image)
      np_image=np_image/255
      return np.expand_dims(np_image,axis=0)

In [9]:
def return_GT_image(json_obj,img):
  with open(json_obj,'r') as f:

    jsnfile=json.load(f)
    GT_obj=jsnfile['objects']
    for obj in GT_obj:
      if obj['classTitle']=="Traffic Sign":
        obj_class_pts = obj['points']['exterior']
        x_0=obj_class_pts[0][0]
        y_0=obj_class_pts[0][1]
        x_1=obj_class_pts[1][0]
        y_1=obj_class_pts[1][1]

        cv2.rectangle(img, (int(x_0), int(y_0)), (int(x_1),int(y_1)),
                              color=(0, 0, 255), 
                              thickness=2)
        plt.imshow(img)
    return img

In [10]:
json_path='/content/drive/MyDrive/model/FO_best.json'

In [ ]:
#draws ground truth bboxs -DONT CHANGE
from keras import models
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
with open(json_path, 'r') as f:
    jsonfile = json.load(f)
    json_obj=jsonfile
#prediction_path="/content/drive/MyDrive/prediction/"
#ground_path="/content/drive/MyDrive/GT/"
final_path="/content/drive/MyDrive/FO_before_aug/"


i=0
for obj in json_obj:

  img_path=return_image_path(obj)
  coordinates=return_coordinates(obj)
  img=cv2.imread(img_path)
  #ground
  img_name=return_image_name(img_path)
  GT_json=return_json_name(img_name)
  GT_img=return_GT_image(GT_json,img) 
  #ground

  img_1,img_0=return_image_size(img)
  
  classification_model=models.load_model("/content/drive/MyDrive/model/classification_model.h5")
  

  #draw predictions
  for bbx in coordinates: 
    test_img=[]
    xcenter,ycenter,width,height,confidence=bbx

    xcenter=xcenter*img_1
    ycenter=ycenter*img_0

    width=width*img_1
    height=height*img_0

    x_min=abs(xcenter-(width/2))
    y_min=abs(ycenter-(height/2))

    x_max=width+x_min
    y_max=height+y_min

    w=int(width)
    h=int(height)

    crop_img = img[int(y_min):int(y_min)+h,int(x_min):int(x_min)+w]

    test_img=prepare_cropped_image(crop_img)
    
    prediction=classification_model.predict(test_img)

    prediction=np.argmax(prediction,axis=1)
    print(prediction)
    cv2.rectangle(img, (int(x_min+w), int(y_min+h)), (int(x_min),int(y_min)),
                              color=(0, 255, 0), 
                              thickness=2)
    sign_label="id"+str(prediction)
    cv2.putText(img,sign_label, (int(x_min), int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (36,255,12), 2)
    
    i+=1
  
  cv2.imwrite(final_path+str(i)+".jpg",img)